In [1]:
!pip install groq python-dotenv numpy tqdm datasets
!pip install ipywidgets


In [2]:
from groq import Groq
from dotenv import load_dotenv
from datasets import load_dataset

import os
from tqdm import tqdm
import re
import random
import pprint

from typing import List, Dict, Any

load_dotenv()
random.seed(0)

client = Groq()
gsm8k_dataset = load_dataset("gsm8k", "main")

gsm8k_train = gsm8k_dataset["train"]
gsm8k_test  = gsm8k_dataset["test"]

In [5]:
def generate_response_using_Llama(
        prompt: str,
        model: str = "llama-3.1-8b-instant"
    ):
    try:
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": "You are a helpful assistant that solves math problems."
                },
                {
                    "role": "user", 
                    "content": prompt
                }
            ],
            model=model,
            temperature=0.3, ### 수정해도 됩니다!
            stream=False
        )
        return chat_completion.choices[0].message.content
    
    except Exception as e:
        print(f"API call error: {str(e)}")
        return None

#### 응답 잘 나오는지 확인해보기

In [6]:
response = generate_response_using_Llama(
    prompt="Hello world!",
)
print(response)

Hello world! I'm here to help with any math problems you might have. What's on your mind? Do you have a specific problem you'd like me to solve, or would you like some help with a particular math concept?


#### GSM8K 데이터셋 확인해보기

In [7]:
print("[Question]")
for l in gsm8k_test['question'][0].split("."):
    print(l)
print("="*100)
print("[Answer]")
print(gsm8k_test['answer'][0])

[Question]
Janet’s ducks lay 16 eggs per day
 She eats three for breakfast every morning and bakes muffins for her friends every day with four
 She sells the remainder at the farmers' market daily for $2 per fresh duck egg
 How much in dollars does she make every day at the farmers' market?
[Answer]
Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.
She makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.
#### 18


#### Util 함수들
- extract_final_answer: LLM의 응답을 parse하여 최종 결과만 추출 (정답과 비교하기 위해)
- run_benchmark_test: 벤치마크 테스트
- save_final_result: 결과물 제출을 위한 함수

In [8]:
### 수정해도 됩니다!
def extract_final_answer(response: str):
    regex = r"(?:Answer:|Model response:)\s*\$?([0-9,]+)\b|([0-9,]+)\s*(meters|cups|miles|minutes)"
    matches = re.finditer(regex, response, re.MULTILINE)
    results = [match.group(1) if match.group(1) else match.group(2).replace(",", "") for match in matches]

    if len(results) == 0:
        additional_regex = r"\$?([0-9,]+)"
        additional_matches = re.finditer(additional_regex, response, re.MULTILINE)
        results.extend([match.group(1).replace(",", "") for match in additional_matches])

    return results[-1] if results else None

In [26]:
### 수정해도 됩니다!
def run_benchmark_test(
        dataset,
        prompt: str,
        model: str = "llama-3.1-8b-instant",
        num_samples: int = 50,
        VERBOSE: bool = False
    ):
    correct = 0
    total   = 0
    results = []

    for i in tqdm(range(min(num_samples, len(dataset)))):
        question = dataset[i]["question"]
        correct_answer = float(re.findall(r'\d+(?:\.\d+)?', dataset[i]["answer"].split('####')[-1])[0])

        response = generate_response_using_Llama(
            prompt=prompt.format(question=question),
            model=model
        )

        if response:
            if VERBOSE:
                print("="*50)
                print(response)
                print("="*50)
            predicted_answer_str = extract_final_answer(response)

            # 수정됨
            if predicted_answer_str:
                try:
                    predicted_answer = float(predicted_answer_str.replace(",", ""))
                except ValueError:
                    if VERBOSE:
                        print(f"Warning: Could not parse answer '{predicted_answer}'")
                    predicted_answer = None

            is_correct = False
            if predicted_answer is not None:
                diff = abs(predicted_answer - correct_answer)
                is_correct = diff < 1e-5

            if is_correct:
                correct += 1
            total += 1
            
            results.append({
                'question': question,
                'correct_answer': correct_answer,
                'predicted_answer': predicted_answer,
                'response': response,
                'correct': is_correct
            })

            if (i + 1) % 5 == 0:
                current_acc = correct/total if total > 0 else 0
                print(f"Progress: [{i+1}/{num_samples}]")
                print(f"Current Acc.: [{current_acc:.2%}]")

    return results, correct/total if total > 0 else 0

In [14]:
def save_final_result(results: List[Dict[str, Any]], accuracy: float, filename: str) -> None:
    result_str = f"====== ACCURACY: {accuracy} ======\n\n"
    result_str += f"[Details]\n"
    
    for idx, result in enumerate(results):
        result_str += f"Question {idx+1}: {result['question']}\n"
        result_str += f"Correct Answer: {result['correct_answer']}\n"
        result_str += f"Predicted Answer: {result['predicted_answer']}\n"
        result_str += f"Correct: {result['correct']}\n\n"
    
    with open(filename, "w", encoding="utf-8") as f:
        f.write(result_str)

#### Direct prompting with few-shot example

In [15]:
def construct_direct_prompt(num_examples: int = 3) -> str:
    train_dataset = gsm8k_train

    sampled_indices = random.sample(
        [i for i in range(len(train_dataset['question']))],
        num_examples
    )

    prompt = "Instruction:\nSolve the following mathematical question and generate ONLY the answer after a tag, 'Answer:' without any rationale.\n"

    for i in range(num_examples):
        cur_question = train_dataset['question'][i]
        cur_answer = train_dataset['answer'][i].split("####")[-1].strip()

        prompt += f"\n[Example {i+1}]\n"
        prompt += f"Question:\n{cur_question}\n"
        prompt += f"Answer:{cur_answer}\n"

    prompt += "\nQuestion:\n{question}\nAnswer:"

    return prompt

In [16]:
### 어떤 방식으로 저장되는지 확인해보세요!
PROMPT = construct_direct_prompt(3)
VERBOSE = False

results, accuracy = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=PROMPT,
    VERBOSE=VERBOSE,
    num_samples=10
)
save_final_result(results, accuracy, "example.txt")

 50%|█████     | 5/10 [00:02<00:02,  2.15it/s]

Progress: [5/10]
Current Acc.: [80.00%]


100%|██████████| 10/10 [00:04<00:00,  2.13it/s]

Progress: [10/10]
Current Acc.: [60.00%]


In [ ]:
# TODO: 0 shot, 3 shot, 5 shot direct prompting을 통해 벤치마크 테스트를 한 후, 각각 direct_prompting_{shot: int}.txt로 저장해주세요!
# 예시: shot이 5인 경우 direct_prompting_5.txt
# 항상 num_samples=50 입니다!

shots = [0, 3, 5]
    
print("\n--- Starting Direct Prompting Benchmarks ---")
for shot in shots:
    filename = f"direct_prompting_{shot}.txt"
    print(f"Processing Direct Prompting {shot}-shot...")
    results, acc = run_benchmark_test(
        dataset=gsm8k_test,
        prompt=construct_direct_prompt(shot),
        VERBOSE=False,
        num_samples=50
    )
    save_final_result(results, acc, filename)


--- Starting Direct Prompting Benchmarks ---
Processing Direct Prompting 0-shot...


 10%|█         | 5/50 [00:01<00:15,  2.82it/s]

Progress: [5/50]
Current Acc.: [80.00%]


 16%|█▌        | 8/50 [00:02<00:15,  2.69it/s]

### Chain-of-Thought prompting with few-shot example
```text
[Question]
Janet’s ducks lay 16 eggs per day
 She eats three for breakfast every morning and bakes muffins for her friends every day with four
 She sells the remainder at the farmers' market daily for $2 per fresh duck egg
 How much in dollars does she make every day at the farmers' market?
====================================================================================================
[Answer]
Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.
She makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.
#### 18
```

[Answer] 아래의 정답을 도출하는 과정을 예시로 달아주면 CoT의 few shot이 되겠죠?

In [ ]:
def construct_CoT_prompt(num_examples: int = 3) -> str:
    train_dataset = gsm8k_train

    sampled_indices = random.sample(
        [i for i in range(len(train_dataset['question']))],
        num_examples
    )
    # CoT prompt instruction
    prompt = "Instruction:\nSolve the following mathematical problems step by step. Show your reasoning and put the final answer after '####'.\n"

    for i, idx in enumerate(sampled_indices):
        cur_question = train_dataset['question'][idx]
        cur_answer = train_dataset['answer'][idx] # Take the FULL answer including reasoning

        prompt += f"\n[Example {i+1}]\n"
        prompt += f"Question:\n{cur_question}\n"
        prompt += f"Answer:\n{cur_answer}\n"

    prompt += "\nQuestion:\n{question}\nAnswer:"

    return prompt

In [ ]:
# TODO: 0 shot, 3 shot, 5 shot CoT prompting을 통해 벤치마크 테스트를 한 후, 각각 CoT_prompting_{shot: int}.txt로 저장해주세요!
# 예시: shot이 5인 경우 CoT_prompting_5.txt
# 항상 num_samples=50 입니다!

print("\n--- Starting CoT Prompting Benchmarks ---")
for shot in shots:
    filename = f"CoT_prompting_{shot}.txt"
    print(f"Processing CoT Prompting {shot}-shot...")
    results, acc = run_benchmark_test(
        dataset=gsm8k_test,
        prompt=construct_CoT_prompt(shot),
        VERBOSE=False,
        num_samples=50
    )
    save_final_result(results, acc, filename)

### Construct your prompt!!

목표: 본인만의 프롬프트를 통해 정답률을 더 끌어올려보기!
- gsm8k의 train 데이터셋에서 예시를 가져온 다음 (자유롭게!)
- 그 예시들에 대한 풀이 과정을 만들어주세요!
- 모든 것들이 자유입니다! Direct Prompting, CoT Prompting을 한 결과보다 정답률만 높으면 돼요.

In [ ]:
### 자유롭게 수정해도 됩니다! 완전히 새로 함수를 만들어도 돼요.
def construct_my_prompt(example_list: List[str], num_examples: int = 3):
    """
    Implements a 'Structured Reasoning' or 'Plan-and-Solve' style prompt.
    Reasons for choice:
    1.  **Decomposition**: Forces the model to break down the problem into 'Key Information' and 'Plan' before calculating.
    2.  **Structure**: The rigid headers ([Goal], [Plan], [Solution]) guide the model's generation process, reducing formatting errors and lost context.
    3.  **Verification**: Implicitly encourages checking steps by separating the plan from the execution.
    """
    train_dataset = gsm8k_train
    
    indices = list(range(len(train_dataset['question'])))
    sampled_indices = random.sample(indices, num_examples)

    # Enhanced Instruction
    prompt = (
        "Instruction:\n"
        "You are an expert mathematician. Solve the math problem by following these steps:\n"
        "1. [Goal]: Briefly state what needs to be calculated.\n"
        "2. [Solution]: Solve the problem step-by-step, showing all arithmetic.\n"
        "3. [Answer]: State the final answer after '####'.\n"
    )

    for i, idx in enumerate(sampled_indices):
        cur_question = train_dataset['question'][idx]
        cur_answer = train_dataset['answer'][idx] 
        # Note: Since the training data doesn't have [Goal] annotation, 
        # we will use the standard CoT answer but wrap it in the structure 
        # to teach the model the *format* even if the few-shot examples aren't perfect 'Plans'.
        # However, to be effective, 0-shot instruction is powerful enough.
        # For the few-shot, we simply provide the Question and the Answer (CoT) styled as [Solution].
        # We simulate the [Goal] by just saying "Calculate the result." to avoid hallucinating wrong goals in examples.
        
        prompt += f"\n[Example {i+1}]\n"
        prompt += f"Question:\n{cur_question}\n"
        prompt += f"[Goal]: Solve the problem.\n"
        prompt += f"[Solution]:\n{cur_answer}\n"

    prompt += "\nQuestion:\n{question}\n"
    prompt += "[Goal]:" # Prompting the model to start filling this in

    return prompt

In [ ]:
# TODO: 만든 0 shot, 3 shot, 5 shot example과 프롬프트를 통해 벤치마크 테스트를 한 후, 각각 My_prompting_{shot: int}.txt로 저장해주세요!
# 예시: shot이 5인 경우 My_prompting_5.txt
# 항상 num_samples=50 입니다!

print("\n--- Starting My Custom Prompting Benchmarks ---")
for shot in shots:
    filename = f"My_prompting_{shot}.txt"
    print(f"Processing My Custom Prompting {shot}-shot...")
    results, acc = run_benchmark_test(
        dataset=gsm8k_test,
        prompt=construct_my_prompt(shot),
        VERBOSE=False,
        num_samples=50
    )
    save_final_result(results, acc, filename)

### 보고서 작성하기
#### 아래의 내용이 포함되면 됩니다!

1. Direct Prompting, CoT Prompting, My Prompting을 0 shot, 3 shot, 5 shot 정답률을 표로 보여주세요!
2. CoT Prompting이 Direct Prompting에 비해 왜 좋을 수 있는지에 대해서 서술해주세요!
3. 본인이 작성한 프롬프트 기법이 CoT에 비해서 왜 더 좋을 수 있는지에 대해서 설명해주세요!
4. 최종적으로, `PROMPTING.md`에 보고서를 작성해주세요!